In [1]:
!nsys profile -s none -o scipy_v1 -f true --stats=true python3 icassp_scipy_v1.py float32 5

Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-3e4f-7590-e563-3688.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-3e4f-7590-e563-3688.qdrep"
Exporting 614938 events: [================================================100%]

Exported successfully to
/tmp/nsys-report-3e4f-7590-e563-3688.sqlite

Generating CUDA API Statistics...
CUDA API Statistics (nanoseconds)




CUDA trace data was not collected.


Generating Operating System Runtime API Statistics...
Operating System Runtime API Statistics (nanoseconds)

Time(%)      Total Time       Calls         Average         Minimum         Maximum  Name                                                                            
-------  --------------  ----------  --------------  --------------  --------------  ----------------------------------------------------------------------------

In [2]:
!nsys profile -s none -o scipy_v1 -f true --stats=true python3 icassp_scipy_v1.py float64 5

Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-0c1a-00f8-bf2d-75e8.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-0c1a-00f8-bf2d-75e8.qdrep"
Exporting 655932 events: [================================================100%]

Exported successfully to
/tmp/nsys-report-0c1a-00f8-bf2d-75e8.sqlite

Generating CUDA API Statistics...
CUDA API Statistics (nanoseconds)




CUDA trace data was not collected.


Generating Operating System Runtime API Statistics...
Operating System Runtime API Statistics (nanoseconds)

Time(%)      Total Time       Calls         Average         Minimum         Maximum  Name                                                                            
-------  --------------  ----------  --------------  --------------  --------------  ----------------------------------------------------------------------------

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,472.3  | 3,469.0     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,470.9  | 3,470.2     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |

In [3]:
!nsys profile -s none -o cupy_v1 -f true --stats=true python3 icassp_cupy_v1.py float32 2000 0

Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-1145-d3d2-4c4c-a970.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-1145-d3d2-4c4c-a970.qdrep"
Exporting 643018 events: [================================================100%]

Exported successfully to
/tmp/nsys-report-1145-d3d2-4c4c-a970.sqlite

Generating CUDA API Statistics...
CUDA API Statistics (nanoseconds)

Time(%)      Total Time       Calls         Average         Minimum         Maximum  Name                                                                            
-------  --------------  ----------  --------------  --------------  --------------  --------------------------------------------------------------------------------
   99.5     84688777543        2001      42323227.2        38788942        44550869  cudaDeviceSynchronize                                      

In [4]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v1.qdrep

Using cupy_v1.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v1.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average      Minimum     Maximum          Name       
 -------  ---------------  ---------  ------------  ----------  ----------  -----------------
   100.0   84,685,961,622      2,001  42,321,819.9  38,788,729  44,549,033  _cupy_lombscargle

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v1.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    99.8   84,864,142,724      2,000   42,432,071.4   38,905,841   44,676,001  cupy_lombscargle_loop
     0.2      167,689,136          1  167,689,136.0  167,689,136  167,689,136  cupy_lombscargle     



In [5]:
!nsys profile -s none -o cupy_v1 -f true python3 icassp_cupy_v1.py float64 2000 0

Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-729d-0530-32b7-a1a9.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-729d-0530-32b7-a1a9.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v1.qdrep"


In [6]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v1.qdrep

Generate SQLite file cupy_v1.sqlite from cupy_v1.qdrep
Exporting 634965 events: [================================================100%]
Using cupy_v1.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v1.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average      Minimum     Maximum          Name       
 -------  ---------------  ---------  ------------  ----------  ----------  -----------------
   100.0  167,442,982,843      2,001  83,679,651.6  81,988,180  84,461,922  _cupy_lombscargle

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v1.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    99.9  167,577,457,463      2,000   83,788,728.7   82,652,130   87,302,519  cupy_lombscargle_loop
 

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,472.3  | 3,469.0     | 1.0      |
| CuPy (Baseline)       |           | 401.1    | 0.911       | 3807.9   |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 3,470.9  | 3,470.2     | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |

In [7]:
!nsys profile -s none -o cupy_v2 -f true python3 icassp_cupy_v2.py float32 2000 0

Registers 30
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-5378-060f-deaa-93e0.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-5378-060f-deaa-93e0.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v2.qdrep"


In [8]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v2.qdrep

Generate SQLite file cupy_v2.sqlite from cupy_v2.qdrep
Exporting 640813 events: [================================================100%]
Using cupy_v2.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v2.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average      Minimum     Maximum          Name       
 -------  ---------------  ---------  ------------  ----------  ----------  -----------------
   100.0   84,745,870,463      2,001  42,351,759.4  38,790,304  44,432,214  _cupy_lombscargle

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v2.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    99.8   84,820,970,119      2,000   42,410,485.1   38,854,687   44,486,923  cupy_lombscargle_loop
 

In [9]:
!nsys profile -s none -o cupy_v2 -f true python3 icassp_cupy_v2.py float64 2000 0

Registers 48
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-c7f1-de75-0213-8773.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-c7f1-de75-0213-8773.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v2.qdrep"


In [10]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v2.qdrep

Generate SQLite file cupy_v2.sqlite from cupy_v2.qdrep
Exporting 642854 events: [================================================100%]
Using cupy_v2.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v2.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average      Minimum     Maximum          Name       
 -------  ---------------  ---------  ------------  ----------  ----------  -----------------
   100.0  168,085,588,817      2,001  84,000,794.0  82,271,576  84,966,117  _cupy_lombscargle

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v2.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    99.9  168,130,033,365      2,000   84,065,016.7   82,841,719   85,031,179  cupy_lombscargle_loop
 

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,140    | 2,139       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,158    | 2,153       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

In [11]:
!nsys profile -s none -o cupy_v3 -f true python3 icassp_cupy_v3.py float32 2000 0

Registers 26
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-74b7-c8bf-0357-f166.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-74b7-c8bf-0357-f166.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v3.qdrep"


In [12]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v3.qdrep

Generate SQLite file cupy_v3.sqlite from cupy_v3.qdrep
Exporting 621943 events: [================================================100%]
Using cupy_v3.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v3.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum              Name           
 -------  ---------------  ---------  -------------  -----------  -----------  -------------------------
   100.0  344,563,605,443      2,001  172,195,704.9  165,755,399  172,874,166  _cupy_lombscargle_float32

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v3.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    99.9  344,686,946,429      2,000  172,343,473.2  165,858,718  242

In [13]:
!nsys profile -s none -o cupy_v3 -f true python3 icassp_cupy_v3.py float64 2000 0

Registers 48
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-005a-b728-83c2-e5d8.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-005a-b728-83c2-e5d8.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v3.qdrep"


In [14]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v3.qdrep

Generate SQLite file cupy_v3.sqlite from cupy_v3.qdrep
Exporting 659003 events: [================================================100%]
Using cupy_v3.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v3.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum              Name           
 -------  ---------------  ---------  -------------  -----------  -----------  -------------------------
   100.0  669,407,968,861      2,001  334,536,716.1  328,522,380  336,758,415  _cupy_lombscargle_float64

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v3.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    99.9  669,316,403,841      2,000  334,658,201.9  329,039,271  336

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,140    | 2,139       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,158    | 2,153       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

In [15]:
!nsys profile -s none -o cupy_v4 -f true python3 icassp_cupy_v4.py float32 2000 0

Registers 32
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-5894-f891-dc73-3353.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-5894-f891-dc73-3353.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v4.qdrep"


In [16]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v4.qdrep

Generate SQLite file cupy_v4.sqlite from cupy_v4.qdrep
Exporting 620992 events: [================================================100%]
Using cupy_v4.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v4.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum             Name           
 -------  ---------------  ---------  -----------  ---------  ---------  -------------------------
   100.0   17,933,945,018      2,001  8,962,491.3  8,575,970  9,618,043  _cupy_lombscargle_float32

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v4.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    99.3   18,126,677,682      2,000    9,063,338.8    8,680,057    9,716,641  cupy_lom

In [ ]:
!nsys profile -s none -o cupy_v4 -f true python3 icassp_cupy_v4.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v4.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,140    | 2,139       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,158    | 2,153       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

In [ ]:
!nvcc --fatbin -std=c++11 --use_fast_math \
    --generate-code arch=compute_70,code=sm_70 \
    --generate-code arch=compute_75,code=sm_75 \
    --generate-code arch=compute_80,code=[sm_80,compute_80] \
    _lombscargle.cu -odir .

In [ ]:
!nsys profile -s none -o cupy_v5 -f true python3 icassp_cupy_v5.py float32 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v5.qdrep

In [ ]:
!nsys profile -s none -o cupy_v5 -f true python3 icassp_cupy_v5.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v5.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,140    | 2,139       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,158    | 2,153       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

In [ ]:
!nvcc --fatbin -std=c++11 --use_fast_math \
    --generate-code arch=compute_70,code=sm_70 \
    --generate-code arch=compute_75,code=sm_75 \
    --generate-code arch=compute_80,code=[sm_80,compute_80] \
    _lombscargle_lb.cu -odir .

In [ ]:
!nsys profile -s none -o cupy_v6 -f true python3 icassp_cupy_v6.py float32 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v6.qdrep

In [ ]:
!nsys profile -s none -o cupy_v6 -f true python3 icassp_cupy_v6.py float64 2000 0

In [ ]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v6.qdrep

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,140    | 2,139       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 2,158    | 2,153       | 1        |
| Numba (Baseline)      |||||
| Numba (User Cache)    |||||
| Numba (Data Type)     |||||
| Numba (Fast Math)     |||||
| Numba (Max Registers) |||||